In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
%load_ext cudf.pandas
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import cuml

# Setting Matplotlib defaults
plt.style.use('seaborn-v0_8')
plt.rc('figure', figsize=(8,5), dpi=150)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=15, titlepad=10)
plt.rc('animation', html='html5')
plt.tight_layout()
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import warnings
warnings.simplefilter('ignore')

pd.set_option('display.max_columns', 500)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/calories-burnt-prediction/calories.csv
/kaggle/input/playground-series-s5e5/sample_submission.csv
/kaggle/input/playground-series-s5e5/train.csv
/kaggle/input/playground-series-s5e5/test.csv


<Figure size 1200x750 with 0 Axes>

In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s5e5/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/playground-series-s5e5/test.csv', index_col='id')
org = pd.read_csv('/kaggle/input/calories-burnt-prediction/calories.csv', index_col='User_ID')
org = org.rename(columns={'Gender': 'Sex'})

In [3]:
train = pd.concat([train, org], ignore_index=True)

# Data Understanding

In [4]:
train.shape

(765000, 8)

In [5]:
train.head(10)

,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,female,38,166.0,61.0,25.0,102.0,40.6,146.0
5,female,26,156.0,56.0,19.0,100.0,40.5,103.0
6,female,21,172.0,73.0,3.0,81.0,38.3,9.0
7,male,46,188.0,94.0,23.0,100.0,40.8,145.0
8,female,33,166.0,63.0,25.0,107.0,40.5,161.0
9,male,65,185.0,88.0,23.0,104.0,41.0,185.0


In [6]:
train.describe()

,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
count,765000.000000,765000.000000,765000.000000,765000.000000,765000.000000,765000.000000,765000.000000
mean,41.447255,174.693126,75.142162,15.423163,95.484672,40.036041,88.307424
std,15.213677,12.854173,14.004122,8.353421,9.452476,0.779863,62.396760
min,20.000000,123.000000,36.000000,1.000000,67.000000,37.100000,1.000000
25%,28.000000,164.000000,63.000000,8.000000,88.000000,39.600000,34.000000
50%,40.000000,174.000000,74.000000,15.000000,95.000000,40.300000,77.000000
75%,52.000000,185.000000,87.000000,23.000000,103.000000,40.700000,136.000000
max,79.000000,222.000000,132.000000,30.000000,128.000000,41.500000,314.000000


In [7]:
train.isnull().sum()

Sex           0
Age           0
Height        0
Weight        0
Duration      0
Heart_Rate    0
Body_Temp     0
Calories      0
dtype: int64

In [8]:
train.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 765000 entries, 0 to 764999
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   Sex         765000 non-null  object
 1   Age         765000 non-null  int64
 2   Height      765000 non-null  float64
 3   Weight      765000 non-null  float64
 4   Duration    765000 non-null  float64
 5   Heart_Rate  765000 non-null  float64
 6   Body_Temp   765000 non-null  float64
 7   Calories    765000 non-null  float64
dtypes: float64(6), int64(1), object(1)
memory usage: 47.4+ MB


# Data Preprocessing

## Reducing memory usage

In [9]:
train['Age'] = train['Age'].astype('int8')
test['Age'] = test['Age'].astype('int8')

In [10]:
num_cols = test.select_dtypes(include='float64').columns
for col in num_cols:
    train[num_cols] = train[num_cols].astype('float32')
    test[num_cols] = test[num_cols].astype('float32')

In [11]:
train.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 765000 entries, 0 to 764999
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   Sex         765000 non-null  object
 1   Age         765000 non-null  int8
 2   Height      765000 non-null  float32
 3   Weight      765000 non-null  float32
 4   Duration    765000 non-null  float32
 5   Heart_Rate  765000 non-null  float32
 6   Body_Temp   765000 non-null  float32
 7   Calories    765000 non-null  float64
dtypes: float32(5), float64(1), int8(1), object(1)
memory usage: 27.7+ MB


In [12]:
mapping = {'male': 1, 'female': 0}

train['Sex'] = train['Sex'].map(mapping).astype('int8')
test['Sex'] = test['Sex'].map(mapping).astype('int8')

In [13]:
features = ['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']

In [14]:
def add_features(df):
    for i, c1 in enumerate(features):
        for c2 in (features[i+1:]):
            m = f'{c1}*{c2}'
            df[m] = df[c1] * df[c2]
            a = f'{c1}+{c2}'
            df[a] = df[c1] + df[c2]
            s = f'{c1}-{c2}'
            df[s] = df[c1] - df[c2]
            rs = f'{c2}-{c1}'
            df[rs] = df[c2] - df[c1]
            d = f'{c1}/{c2}'
            df[d] = df[c1] / (df[c2] + 1e-5)
            rd = f'{c2}/{c1}'
            df[rd] = df[c2] / (df[c1] + 1e-5)
    return df

In [15]:
def add_statistical_features(df, features):
    df_new = df.copy()
    df_new["row_mean"] = df[features].mean(axis=1)
    df_new["row_std"] = df[features].std(axis=1)
    df_new["row_max"] = df[features].max(axis=1)
    df_new["row_min"] = df[features].min(axis=1)
    df_new["row_median"] = df[features].median(axis=1)
    
    return df_new

In [16]:
def feature_engineer(df):
    Max_Heart_Rate = 220 - df['Age']
    df['HR_Ratio'] = df['Heart_Rate'] / Max_Heart_Rate
    df['HR_Reserve'] = (df['Heart_Rate'] / Max_Heart_Rate) * df['Duration']

    df['MET_Estimate'] = (df['Heart_Rate'] / Max_Heart_Rate) * 15
    df['MET_Calories_Rate'] = df['MET_Estimate'] * 3.5 * df['Weight'] / 200
    
    df['MET_Calories_Total'] = df['MET_Calories_Rate'] * df['Duration']
    
    df['Exercise_Intensity_Index'] = df['Heart_Rate'] * df['Body_Temp'] / (df['Weight'] * df['Age'])
    df['Intensity'] = df['Heart_Rate']  * df['Duration'] / 60

    df['Exertion_score'] = df['Duration'] * df['Heart_Rate'] * df['Body_Temp']
    
    df['BMI'] = df['Weight'] / (df['Height']/100) ** 2

    for col in ['Duration', 'Heart_Rate', 'Body_Temp']:
        df[f'Sqrt_{col}'] = np.sqrt(df[col])
        df[f'cube_{col}'] = df[col] ** 3
        
    df['EPOC_Factor'] = df['HR_Ratio']**2 * np.log1p(df['Duration']/10)
    df['Thermal_Load'] = df['Body_Temp'] * df['Duration'] * df['HR_Ratio']

    df['CB_Proxy'] = np.where(
        df['Sex'] == 'female',
        df['Duration'] * (0.4472 * df['Heart_Rate'] - 0.1263 * df['Weight'] + 0.074 * df['Age'] -  55.0969) / 4.184,
        df['Duration'] * (0.6309 * df['Heart_Rate'] - 0.1988 * df['Weight'] + 0.2017 * df['Age'] -  55.0969) / 4.184,
    )

    bins = [0, 10, 20, 30]
    Duration_Bin = pd.cut(df['Duration'], bins=bins, labels=[f"{bins[i]}_{bins[i+1]}" for i in range(len(bins)-1)])

    for bin_label in Duration_Bin.unique():
        df[f'HR_DurBin_{bin_label}'] = np.where(Duration_Bin == bin_label, df['Heart_Rate'], np.nan)
        df[f'BT_DurBin_{bin_label}'] = np.where(Duration_Bin == bin_label, df['Body_Temp'], np.nan)

    return df

In [17]:
train = add_features(train)
train = add_statistical_features(train, features=['Height', 'Weight','Heart_Rate', 'Body_Temp'])
train = feature_engineer(train)

test = add_features(test)
test = add_statistical_features(test, features=['Height', 'Weight','Heart_Rate', 'Body_Temp'])
test = feature_engineer(test)

In [18]:
common_cols = [col for col in test.columns if col in train.columns and col != 'Calories']
train = train[common_cols + ['Calories']]
test = test[common_cols]

In [19]:
X = train.copy()
y = X.pop('Calories')
y = np.log1p(y)

X_test = test.copy()

In [20]:
corr_matrix = pd.DataFrame(X).corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
X = X.drop(columns=to_drop)
X_test = X_test.drop(columns=to_drop)

In [21]:
X.head()

,Sex,Age,Height,Duration,Heart_Rate,Body_Temp,Age+Height,Age-Height,Height/Age,Age*Duration,Age+Duration,Age-Duration,Age/Duration,Duration/Age,Heart_Rate/Age,Height-Weight,Height+Duration,Height-Duration,Height/Duration,Height*Heart_Rate,Height/Heart_Rate,Duration-Heart_Rate,MET_Calories_Rate,Exercise_Intensity_Index,BMI,cube_Duration,CB_Proxy,HR_DurBin_0_10,BT_DurBin_0_10,HR_DurBin_10_20,BT_DurBin_10_20,HR_DurBin_20_30,BT_DurBin_20_30
0,1,36,189.0,26.0,101.0,41.000000,225.0,-153.0,5.249999,936.0,62.0,10.0,1.384615,0.722222,2.805555,107.0,215.0,163.0,7.269228,19089.0,1.871287,-75.0,11.815354,1.402778,22.955685,17576.0,-2.587548,NaN,NaN,NaN,NaN,101.0,41.000000
1,0,64,163.0,8.0,85.0,39.700001,227.0,-99.0,2.546875,512.0,72.0,56.0,7.999990,0.125000,1.328125,103.0,171.0,155.0,20.374975,13855.0,1.917647,-77.0,8.581730,0.878776,22.582710,512.0,-0.936135,85.0,39.700001,NaN,NaN,NaN,NaN
2,0,51,161.0,7.0,84.0,39.799999,212.0,-110.0,3.156862,357.0,58.0,44.0,7.285704,0.137255,1.647059,97.0,168.0,154.0,22.999968,13524.0,1.916667,-77.0,8.350297,1.024265,24.690405,343.0,-7.591922,84.0,39.799999,NaN,NaN,NaN,NaN
3,1,20,192.0,25.0,105.0,40.700001,212.0,-172.0,9.599995,500.0,45.0,-5.0,0.800000,1.249999,5.249997,102.0,217.0,167.0,7.679997,20160.0,1.828571,-80.0,12.403124,2.374167,24.414062,15625.0,-16.195023,NaN,NaN,NaN,NaN,105.0,40.700001
4,0,38,166.0,25.0,102.0,40.599998,204.0,-128.0,4.368420,950.0,63.0,13.0,1.519999,0.657895,2.684210,105.0,191.0,141.0,6.639997,16932.0,1.627451,-77.0,8.974038,1.786540,22.136740,15625.0,28.637061,NaN,NaN,NaN,NaN,102.0,40.599998


In [22]:
X.columns

Index(['Sex', 'Age', 'Height', 'Duration', 'Heart_Rate', 'Body_Temp',
       'Age+Height', 'Age-Height', 'Height/Age', 'Age*Duration',
       'Age+Duration', 'Age-Duration', 'Age/Duration', 'Duration/Age',
       'Heart_Rate/Age', 'Height-Weight', 'Height+Duration', 'Height-Duration',
       'Height/Duration', 'Height*Heart_Rate', 'Height/Heart_Rate',
       'Duration-Heart_Rate', 'MET_Calories_Rate', 'Exercise_Intensity_Index',
       'BMI', 'cube_Duration', 'CB_Proxy', 'HR_DurBin_0_10', 'BT_DurBin_0_10',
       'HR_DurBin_10_20', 'BT_DurBin_10_20', 'HR_DurBin_20_30',
       'BT_DurBin_20_30'],
      dtype='object')

In [23]:
from sklearn.model_selection import KFold
from cuml.metrics import mean_squared_log_error
from xgboost import XGBRegressor

n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=100)

oof_xgb = np.zeros(len(y))
test_xgb = np.zeros(len(X_test))

params = {
    'n_estimators': 3000, 
    'learning_rate': 0.012429613949090264, 
    'max_depth': 10, 
    'min_child_weight': 7, 
    'alpha': 0.031799792818970524, 
    'reg_lambda': 0.3580931274078332, 
    'subsample': 0.8416119186946701, 
    'colsample_bytree': 0.5066402912386329, 
    'gamma': 1.037011350094906e-08,
    'eval_metric': 'rmse',
    'random_state': 100,
    'early_stopping_rounds': 100,
    'tree_method': 'gpu_hist',
    'objective':'reg:squarederror',
    'enable_categorical': 'True',
    'device':'cuda'
}

for fold, (train_index, valid_index) in enumerate(kf.split(X, y), start=1):
        X_train, X_valid = X.iloc[train_index].copy(), X.iloc[valid_index].copy()
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
         
        xgb = XGBRegressor(**params).fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=0)
    
        valid_pred = xgb.predict(X_valid)
        oof_xgb[valid_index] = valid_pred
        test_xgb += xgb.predict(X_test) / n_folds
    
        fold_rmsle = np.sqrt(mean_squared_log_error(np.expm1(y_valid), np.expm1(valid_pred)))
        print(f"Fold {fold} RMSE: {fold_rmsle:.6f}")

test_xgb = np.expm1(test_xgb)
test_xgb = np.clip(test_xgb, 1, 314)

overall_rmsle = np.sqrt(mean_squared_log_error(np.expm1(y), np.expm1(oof_xgb)))
print(f"\nOverall OOF RMSLE: {overall_rmsle:.6f}")   

Fold 1 RMSE: 0.058828
Fold 2 RMSE: 0.058994
Fold 3 RMSE: 0.059713
Fold 4 RMSE: 0.059864
Fold 5 RMSE: 0.058828

Overall OOF RMSLE: 0.059247


In [24]:
oof_xgb_2 = np.zeros(len(y))
test_xgb_2 = np.zeros(len(X_test))

params = {
    'max_depth': 9,
    'colsample_bytree': 0.7,
    'subsample': 0.9,
    'n_estimators': 3000,
    'learning_rate': 0.01,
    'gamma': 0.01,
    'max_delta_step': 2,
    'eval_metric': 'rmse',
    'random_state': 100,
    'early_stopping_rounds': 100,
    'tree_method': 'gpu_hist',
    'objective':'reg:squarederror',
    'device':'cuda'
    }

for fold, (train_index, valid_index) in enumerate(kf.split(X, y), start=1):
        X_train, X_valid = X.iloc[train_index].copy(), X.iloc[valid_index].copy()
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
         
        xgb_2 = XGBRegressor(**params).fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=0)
    
        valid_pred = xgb_2.predict(X_valid)
        oof_xgb_2[valid_index] = valid_pred
        test_xgb_2 += xgb_2.predict(X_test) / n_folds
    
        fold_rmsle = np.sqrt(mean_squared_log_error(np.expm1(y_valid), np.expm1(valid_pred)))
        print(f"Fold {fold} RMSE: {fold_rmsle:.6f}")

test_xgb_2 = np.expm1(test_xgb_2)
test_xgb_2 = np.clip(test_xgb_2, 1, 314)

overall_rmsle = np.sqrt(mean_squared_log_error(np.expm1(y), np.expm1(oof_xgb_2)))
print(f"\nOverall OOF RMSLE: {overall_rmsle:.6f}")   

Fold 1 RMSE: 0.058681
Fold 2 RMSE: 0.058669
Fold 3 RMSE: 0.059601
Fold 4 RMSE: 0.059884
Fold 5 RMSE: 0.058880

Overall OOF RMSLE: 0.059145


In [25]:
X = train.copy()
y = X.pop('Calories')
y = np.log1p(y) 
X['Sex'] = X['Sex'].astype('category')

X_test = test.copy()
X_test['Sex'] = X_test['Sex'].astype('category')

In [26]:
corr_matrix = pd.DataFrame(X).corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
X = X.drop(columns=to_drop)
X_test = X_test.drop(columns=to_drop)

In [27]:
from catboost import CatBoostRegressor
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import StratifiedKFold

bins = KBinsDiscretizer(n_bins=12, encode='ordinal', strategy='kmeans')
duration_bins = bins.fit_transform(train[['Duration']]).astype(int).flatten()

skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=100)
oof_cat = np.zeros(len(y))
test_cat = np.zeros(len(X_test))

params = {
    'iterations': 3000, 
    'learning_rate': 0.03813511357889326, 
    'depth': 9, 
    'l2_leaf_reg': 5.101121281815585, 
    'bagging_temperature': 0.6263562727371863, 
    'random_strength': 0.6489211175248135,
    'loss_function': 'RMSE',
    'cat_features': ['Sex'],
    'eval_metric': 'RMSE',
    'early_stopping_rounds': 100,
    'verbose': 0,
    'random_seed': 100,
    'task_type': 'GPU'   
}

for fold, (train_index, valid_index) in enumerate(skf.split(X, duration_bins), start=1):
        X_train, X_valid = X.iloc[train_index].copy(), X.iloc[valid_index].copy()
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

        cat = CatBoostRegressor(**params).fit(X_train, y_train, eval_set=[(X_valid, y_valid)], use_best_model=True)
    
        valid_pred = cat.predict(X_valid)
        oof_cat[valid_index] = valid_pred
        test_cat += cat.predict(X_test) / n_folds
    
        fold_rmsle = np.sqrt(mean_squared_log_error(np.expm1(y_valid), np.expm1(valid_pred)))
        print(f"Fold {fold} RMSE: {fold_rmsle:.6f}")

test_cat = np.expm1(test_cat)
test_cat = np.clip(test_cat, 1, 314)

overall_rmsle = np.sqrt(mean_squared_log_error(np.expm1(y), np.expm1(oof_cat)))
print(f"\nOverall OOF RMSLE: {overall_rmsle:.6f}")

Fold 1 RMSE: 0.059895
Fold 2 RMSE: 0.058818
Fold 3 RMSE: 0.058974
Fold 4 RMSE: 0.058583
Fold 5 RMSE: 0.059299

Overall OOF RMSLE: 0.059115


In [28]:
from catboost import CatBoostRegressor
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import StratifiedKFold

bins = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')
duration_bins = bins.fit_transform(train[['Duration']]).astype(int).flatten()

skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=100)
oof_cat_2 = np.zeros(len(y))
test_cat_2 = np.zeros(len(X_test))

params = {
    'iterations': 2500, 
    'learning_rate': 0.02, 
    'depth': 10, 
    'l2_leaf_reg': 3, 
    'loss_function': 'RMSE',
    'cat_features': ['Sex'],
    'eval_metric': 'RMSE',
    'early_stopping_rounds': 100,
    'verbose': 0,
    'random_seed': 100,
    'task_type': 'GPU'   
}

for fold, (train_index, valid_index) in enumerate(skf.split(X, duration_bins), start=1):
        X_train, X_valid = X.iloc[train_index].copy(), X.iloc[valid_index].copy()
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

        cat_2 = CatBoostRegressor(**params).fit(X_train, y_train, eval_set=[(X_valid, y_valid)], use_best_model=True)
    
        valid_pred = cat_2.predict(X_valid)
        oof_cat_2[valid_index] = valid_pred
        test_cat_2 += cat_2.predict(X_test) / n_folds
    
        fold_rmsle = np.sqrt(mean_squared_log_error(np.expm1(y_valid), np.expm1(valid_pred)))
        print(f"Fold {fold} RMSE: {fold_rmsle:.6f}")

test_cat_2 = np.expm1(test_cat_2)
test_cat_2 = np.clip(test_cat_2, 1, 314)

overall_rmsle = np.sqrt(mean_squared_log_error(np.expm1(y), np.expm1(oof_cat_2)))
print(f"\nOverall OOF RMSLE: {overall_rmsle:.6f}")

Fold 1 RMSE: 0.059004
Fold 2 RMSE: 0.059999
Fold 3 RMSE: 0.058765
Fold 4 RMSE: 0.058966
Fold 5 RMSE: 0.059089

Overall OOF RMSLE: 0.059166


In [29]:
from sklearn.linear_model import BayesianRidge

meta_train = np.column_stack((oof_xgb, oof_xgb_2, oof_cat, oof_cat_2))
meta_test = np.column_stack((test_xgb, test_xgb_2, test_cat, test_cat_2))

meta_model = BayesianRidge(n_iter=2000).fit(meta_train, y)

final_preds = meta_model.predict(meta_test)

In [30]:
sub = pd.read_csv('/kaggle/input/playground-series-s5e5/sample_submission.csv')
sub['Calories'] = final_preds
sub.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")
print(sub.head(5))

Your submission was successfully saved!
       id    Calories
0  750000   27.403653
1  750001  107.915200
2  750002   86.775778
3  750003  125.250797
4  750004   75.702155
